In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle

# Import modules tự tạo
from nlp_toolkit import (
    DataLoader,
    DataCleaner,
    TextParser,
    BagOfWordsEncoder,
    TfidfEncoder,
    Word2VecEncoder,
    FastTextEncoder,
    CharNgramEncoder,
    PhoBertEncoder,
    convert_json_to_parquet,
    save_results_to_pickle
)

from ml_engine import (
    MLTrainer,
    PerformanceEvaluator,
    ExperimentLogger
)

print("✅ Import thành công!")

✅ Import thành công!


In [2]:
# Đường dẫn đến file dữ liệu
DATA_PATH = 'data.parquet'

# Khởi tạo data loader
loader = DataLoader(DATA_PATH)

# Xử lý dữ liệu
labeled_data, unlabeled_data = loader.process_dataset(sample_limit=None)

print(f"\n✅ Hoàn thành tiền xử lý dữ liệu!")
print(f"Số mẫu có label: {len(labeled_data)}")
print(f"Số mẫu không label: {len(unlabeled_data)}")

BẮT ĐẦU XỬ LÝ DỮ LIỆU
Tổng số comment ban đầu: 800
Sau khi loại bỏ trùng lặp: 758
Sau khi lọc độ dài: 652
Đang chuẩn hóa văn bản:


Xử lý: 100%|██████████| 652/652 [00:00<00:00, 54023.83it/s]

Mẫu có label: 652
Mẫu không có label: 0

✅ Hoàn thành tiền xử lý dữ liệu!
Số mẫu có label: 652
Số mẫu không label: 0


In [3]:
# Tách dữ liệu thành texts và labels
texts = [item[0] for item in labeled_data]
labels = [item[1] for item in labeled_data]

# Chia tập train/test
X_train_text, X_test_text, y_train, y_test = train_test_split(
    texts, labels, 
    test_size=0.2, 
    random_state=42,
    stratify=labels
)

print(f"Training set: {len(X_train_text)} mẫu")
print(f"Test set: {len(X_test_text)} mẫu")
print(f"\nPhân bố nhãn trong tập train:")
print(pd.Series(y_train).value_counts())

Training set: 521 mẫu
Test set: 131 mẫu

Phân bố nhãn trong tập train:
happy       147
surprise    139
toxic       110
negative     83
fear         42
Name: count, dtype: int64


In [4]:
# Khởi tạo các encoder
encoders = {
    'BoW': BagOfWordsEncoder(max_vocab=1000, ngram_config=(1, 2)),
    'TF-IDF': TfidfEncoder(max_vocab=1000, ngram_config=(1, 3)),
    'TF-IDF-SVD': TfidfEncoder(max_vocab=1000, ngram_config=(1, 3), 
                               use_reduction=True, reduced_dims=100),
    'Word2Vec': Word2VecEncoder(vector_size=100, window_size=5, epochs=10),
    'FastText': FastTextEncoder(vector_size=100, window_size=5, epochs=10),
    'CharNgram': CharNgramEncoder(max_features=5000, ngram_range=(2, 5)),
    'PhoBERT': PhoBertEncoder(model_name='vinai/phobert-base', pooling='mean')
}

print("✅ Đã khởi tạo các encoder:")
for name in encoders.keys():
    print(f"  - {name}")

Đang load PhoBERT model: vinai/phobert-base...


✅ Đã load PhoBERT trên cpu
✅ Đã khởi tạo các encoder:
  - BoW
  - TF-IDF
  - TF-IDF-SVD
  - Word2Vec
  - FastText
  - CharNgram
  - PhoBERT


In [5]:
# Khởi tạo trainer và logger
trainer = MLTrainer()
logger = ExperimentLogger()
evaluator = PerformanceEvaluator()

print("✅ Đã khởi tạo ML Trainer và Experiment Logger")
print(f"\nCác thuật toán sẽ được đánh giá:")
for algo_name in trainer.algorithms.keys():
    print(f"  - {algo_name}")

✅ Đã khởi tạo ML Trainer và Experiment Logger

Các thuật toán sẽ được đánh giá:
  - MultinomialNB
  - LogisticRegressor
  - RidgeClassifier
  - DecisionTree
  - RandomForest
  - AdaBoost
  - KNN
  - SVM-Linear
  - SVM-RBF


In [6]:
all_results = {}

for encoder_name, encoder in encoders.items():
    print(f"\n{'='*80}")
    print(f"🎯 BẮT ĐẦU XỬ LÝ ENCODER: {encoder_name}")
    print(f"{'='*80}")
    
    try:
        # Huấn luyện encoder và transform dữ liệu
        print(f"\n📝 Đang huấn luyện {encoder_name}...")
        X_train = encoder.fit_transform(X_train_text)
        X_test = encoder.transform(X_test_text)
        
        print(f"✅ Hoàn thành! Shape: Train={X_train.shape}, Test={X_test.shape}")
        
        # Chạy đánh giá với tất cả các thuật toán
        results = trainer.run_evaluation(
            X_train, X_test,
            y_train, y_test,
            y_train, y_test,
            encoder_name
        )
        
        # Lưu kết quả
        all_results[encoder_name] = results
        
        # Log các thử nghiệm
        for model_name, metrics in results.items():
            logger.log_experiment(encoder_name, model_name, metrics)
        
    except Exception as e:
        print(f"❌ LỖI khi xử lý {encoder_name}: {str(e)}")
        all_results[encoder_name] = {}

print(f"\n{'='*80}")
print("✅ HOÀN THÀNH TẤT CẢ CÁC THỬ NGHIỆM!")
print(f"{'='*80}")


🎯 BẮT ĐẦU XỬ LÝ ENCODER: BoW

📝 Đang huấn luyện BoW...
✅ Hoàn thành! Shape: Train=(521, 366), Test=(131, 366)

ĐÁNH GIÁ VỚI FEATURE: BoW

MultinomialNB:
  Accuracy:  0.7786
  F1-Score:  0.7780
  Precision: 0.7990
  Recall:    0.7786
              precision    recall  f1-score   support

        fear       0.75      0.60      0.67        10
       happy       0.72      0.92      0.81        37
    negative       0.62      0.71      0.67        21
    surprise       0.88      0.83      0.85        35
       toxic       0.95      0.64      0.77        28

    accuracy                           0.78       131
   macro avg       0.78      0.74      0.75       131
weighted avg       0.80      0.78      0.78       131


LogisticRegressor:
  Accuracy:  0.8092
  F1-Score:  0.8064
  Precision: 0.8082
  Recall:    0.8092
              precision    recall  f1-score   support

        fear       0.80      0.80      0.80        10
       happy       0.84      0.84      0.84        37
    negative  

BERT Encoding: 100%|██████████| 9/9 [00:04<00:00,  2.05it/s]


✅ Hoàn thành! Shape: Train=(521, 768), Test=(131, 768)

ĐÁNH GIÁ VỚI FEATURE: PhoBERT

MultinomialNB:
  Accuracy:  0.3740
  F1-Score:  0.3567
  Precision: 0.4477
  Recall:    0.3740
              precision    recall  f1-score   support

        fear       0.20      0.90      0.32        10
       happy       0.33      0.08      0.13        37
    negative       0.29      0.33      0.31        21
    surprise       0.54      0.60      0.57        35
       toxic       0.69      0.32      0.44        28

    accuracy                           0.37       131
   macro avg       0.41      0.45      0.35       131
weighted avg       0.45      0.37      0.36       131


LogisticRegressor:
  Accuracy:  0.7328
  F1-Score:  0.7296
  Precision: 0.7319
  Recall:    0.7328
              precision    recall  f1-score   support

        fear       0.60      0.60      0.60        10
       happy       0.74      0.86      0.80        37
    negative       0.65      0.62      0.63        21
    surprise

In [7]:
# In tóm tắt các thử nghiệm
logger.print_summary()

# Lấy model tốt nhất
best_model, best_name, best_score = trainer.get_champion_model()

print(f"\n{'='*80}")
print("🏆 MODEL TỐT NHẤT TOÀN CỤC")
print(f"{'='*80}")
print(f"Tên: {best_name}")
print(f"Accuracy: {best_score:.4f}")
print(f"Model: {type(best_model).__name__}")


TÓM TẮT CÁC THỬ NGHIỆM

[1] BoW + MultinomialNB
    Accuracy: 0.7786
    F1-Score: 0.7780

[2] BoW + LogisticRegressor
    Accuracy: 0.8092
    F1-Score: 0.8064

[3] BoW + RidgeClassifier
    Accuracy: 0.7481
    F1-Score: 0.7405

[4] BoW + DecisionTree
    Accuracy: 0.5649
    F1-Score: 0.5641

[5] BoW + RandomForest
    Accuracy: 0.6565
    F1-Score: 0.6293

[6] BoW + AdaBoost
    Accuracy: 0.5496
    F1-Score: 0.5350

[7] BoW + KNN
    Accuracy: 0.5954
    F1-Score: 0.5769

[8] BoW + SVM-Linear
    Accuracy: 0.7863
    F1-Score: 0.7822

[9] BoW + SVM-RBF
    Accuracy: 0.7252
    F1-Score: 0.7214

[10] TF-IDF + MultinomialNB
    Accuracy: 0.7786
    F1-Score: 0.7768

[11] TF-IDF + LogisticRegressor
    Accuracy: 0.7634
    F1-Score: 0.7420

[12] TF-IDF + RidgeClassifier
    Accuracy: 0.7634
    F1-Score: 0.7585

[13] TF-IDF + DecisionTree
    Accuracy: 0.5344
    F1-Score: 0.5330

[14] TF-IDF + RandomForest
    Accuracy: 0.6947
    F1-Score: 0.6598

[15] TF-IDF + AdaBoost
    Accura

In [8]:
# Tạo bảng so sánh
comparison_data = []

for encoder_name, results in all_results.items():
    for model_name, metrics in results.items():
        if metrics['accuracy'] > 0:
            comparison_data.append({
                'Encoder': encoder_name,
                'Model': model_name,
                'Accuracy': metrics['accuracy'],
                'F1-Score': metrics['f1_score'],
                'Precision': metrics['precision'],
                'Recall': metrics['recall']
            })

# Tạo DataFrame và sắp xếp theo Accuracy
df_results = pd.DataFrame(comparison_data)
df_results = df_results.sort_values('Accuracy', ascending=False)

print("\n📊 BẢNG SO SÁNH CHI TIẾT:")
print(df_results.to_string(index=False))

# Top 10 kết quả tốt nhất
print("\n🌟 TOP 10 KẾT QUẢ TỐT NHẤT:")
print(df_results.head(10).to_string(index=False))


📊 BẢNG SO SÁNH CHI TIẾT:
   Encoder             Model  Accuracy  F1-Score  Precision   Recall
 CharNgram        SVM-Linear  0.900763  0.903089   0.908077 0.900763
 CharNgram           SVM-RBF  0.877863  0.869246   0.885801 0.877863
 CharNgram   RidgeClassifier  0.870229  0.867509   0.873286 0.870229
 CharNgram     MultinomialNB  0.862595  0.864206   0.878546 0.862595
 CharNgram               KNN  0.854962  0.848682   0.858293 0.854962
 CharNgram LogisticRegressor  0.847328  0.839658   0.857591 0.847328
       BoW LogisticRegressor  0.809160  0.806375   0.808196 0.809160
TF-IDF-SVD           SVM-RBF  0.801527  0.795259   0.797054 0.801527
       BoW        SVM-Linear  0.786260  0.782231   0.783495 0.786260
TF-IDF-SVD LogisticRegressor  0.786260  0.768751   0.802619 0.786260
   PhoBERT        SVM-Linear  0.778626  0.776712   0.778032 0.778626
    TF-IDF     MultinomialNB  0.778626  0.776835   0.778517 0.778626
TF-IDF-SVD        SVM-Linear  0.778626  0.773762   0.776065 0.778626
       B

In [11]:
# Chuẩn bị dữ liệu để lưu
# Lấy encoder tốt nhất để lưu cùng model
best_encoder_name = best_name.split(' + ')[0]
best_encoder = encoders.get(best_encoder_name, None)

export_data = {
    'all_results': all_results,
    'best_model': best_model,
    'best_encoder': best_encoder,
    'best_model_name': best_name,
    'best_encoder_name': best_encoder_name,
    'best_accuracy': best_score,
    'results_dataframe': df_results,
    'experiment_log': logger.experiments
}

# Lưu vào file pickle
save_results_to_pickle(export_data, 'training_results.pkl')

print("✅ Đã lưu kết quả vào training_results.pkl")
print(f"Model tốt nhất: {best_name}")
print(f"Encoder tốt nhất: {best_encoder_name}")

Đã lưu kết quả vào training_results.pkl
✅ Đã lưu kết quả vào training_results.pkl
Model tốt nhất: CharNgram + SVM-Linear
Encoder tốt nhất: CharNgram


In [13]:
# Lấy encoder tốt nhất
best_encoder_name = best_name.split(' + ')[0]
best_encoder_obj = encoders.get(best_encoder_name, None)

print(f"🔍 Kiểm tra encoder:")
print(f"  - Tên encoder: {best_encoder_name}")
print(f"  - Encoder object: {type(best_encoder_obj).__name__ if best_encoder_obj else 'None'}")
print(f"  - Model type: {type(best_model).__name__}")
print(f"  - Model expects: {best_model.n_features_in_} features")

if best_encoder_obj is None:
    print(f"\n⚠️ Không tìm thấy encoder {best_encoder_name}")
    print("Các encoder có sẵn:", list(encoders.keys()))
else:
    # Test transform một text để xem shape
    test_text = "test sample"
    cleaned_test = DataCleaner.sanitize_content(test_text)
    try:
        test_features = best_encoder_obj.transform([cleaned_test])
        print(f"  - Encoder output shape: {test_features.shape}")
        
        # Kiểm tra shape có khớp không
        if test_features.shape[1] != best_model.n_features_in_:
            print(f"\n❌ LỖI: Shape không khớp!")
            print(f"   Encoder tạo {test_features.shape[1]} features")
            print(f"   Model cần {best_model.n_features_in_} features")
            print(f"\n💡 Có thể encoder '{best_encoder_name}' trong dict không phải encoder đã train model")
        else:
            print(f"\n✅ Shape khớp! Có thể dự đoán.")
            
            # Hàm dự đoán
            def predict_text(text):
                cleaned = DataCleaner.sanitize_content(text)
                features = best_encoder_obj.transform([cleaned])
                prediction = best_model.predict(features)
                
                if hasattr(best_model, 'predict_proba'):
                    proba = best_model.predict_proba(features)
                    return prediction[0], proba[0]
                else:
                    return prediction[0], None

            # Test
            test_samples = [
                "This is a great video, I love it!",
                "Terrible content, waste of time",
                "Not bad, could be better"
            ]

            print("\n🔮 DỰ ĐOÁN VỚI MODEL TỐT NHẤT:")
            print(f"Model: {best_name}")
            print(f"Encoder: {best_encoder_name}\n")

            for text in test_samples:
                pred, proba = predict_text(text)
                print(f"Text: {text}")
                print(f"Prediction: {pred}")
                if proba is not None:
                    print(f"Confidence: {max(proba):.4f}")
                print("-" * 50)
                
    except Exception as e:
        print(f"\n❌ Lỗi khi test transform: {e}")

🔍 Kiểm tra encoder:
  - Tên encoder: CharNgram
  - Encoder object: CharNgramEncoder
  - Model type: LinearSVC
  - Model expects: 768 features
  - Encoder output shape: (1, 3632)

❌ LỖI: Shape không khớp!
   Encoder tạo 3632 features
   Model cần 768 features

💡 Có thể encoder 'CharNgram' trong dict không phải encoder đã train model
